In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import numpy
import xgboost as xgb
import statistics as st
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import random

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_df = pd.read_csv("train.csv")

In [3]:
valid_df = pd.read_csv("validation.csv")

In [ ]:
test_df = pd.read_csv("test.csv")

In [4]:
def test_results(threshold, y_p, real_clicks, payprice):
    
    pred_clicks = y_p > threshold # predicted clicks with threshold
    impressions = sum(pred_clicks)
    clicked = sum((pred_clicks + real_clicks) == 2)
    total_cost = sum(valid_df.payprice[clicks_0 == 1])
    
    ctr = clicked/impressions
    cpc = total_cost/clicked
    
    
    print("CTR is ", ctr, " impressions ", impressions, " clicks ", clicked, " total cost ", total_cost, " CPC ", cpc)
    
    return pred_clicks

In [5]:
def test_results_budget(clicks, payprice, real_clicks):
    
    sum_ones = 0
    spent = 0
    click_total = 0

    for i in range(0,len(clicks),3):
        if (clicks[i] == 1) & (spent < 6250):
            click_total = click_total + 1
            spent = spent + payprice[i]
            if real_clicks[i] == 1:
                sum_ones = sum_ones + 1

        if spent > 6250:
            break

    cpc = spent/sum_ones
    ctr_1 = sum_ones/click_total
    print("Sum payed for the clicked ones ", spent, " successful clicks ", sum_ones, " out of ", click_total)
    print("Cost per Click ", cpc)
    print("CTR ", ctr_1)

## clean data

In slotformat map Na to 10 and in adexchange map null tu 0

In [6]:
for index, c in train_df.iterrows():  
    if c.slotformat in "Na":
        train_df.set_value(index, 'slotformat', 10)
        
d = {'1': 1, '2': 2, '3': 3 ,'4': 4, 'null': 0}
train_df['adexchange']=train_df['adexchange'].map(d)

In [7]:
for index, c in valid_df.iterrows():  
    if c.slotformat in "Na":
        valid_df.set_value(index, 'slotformat', 10)

d = {'1': 1, '2': 2, '3': 3 ,'4': 4, 'null': 0}
valid_df['adexchange']=valid_df['adexchange'].map(d)

In [ ]:
for index, c in test_df.iterrows():  
    if c.slotformat in "Na":
        test_df.set_value(index, 'slotformat', 10)

d = {'1': 1, '2': 2, '3': 3 ,'4': 4, 'null': 0}
test_df['adexchange']=test_df['adexchange'].map(d)

In [9]:
#Define features
features = ["weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange"]
# Prepare X and y

train_df['area'] = train_df['slotwidth']*train_df['slotheight']


X_train = train_df[features]
y_train = train_df['click']

In [10]:
# Train the model to estimate pCTR
clf = xgb.XGBRegressor(n_estimators=100, max_depth = 3, learning_rate=0.5, objective= 'binary:logistic')
clf.fit(X_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.5, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [11]:
# preprocess validation set
valid_df['area'] = valid_df['slotwidth']*valid_df['slotheight']

X_valid = valid_df[features]

In [12]:
## predicts probabilities of being clicked for validation
y_p_valid = clf.predict(X_valid)

In [13]:
avgCTR = sum(y_p_valid)/len(y_p_valid)
print("Average CTR ", avgCTR)

Average CTR  0.000755362039884


In [14]:
def calculate_bid_q(base_bid, y_pred, avgCTR):
    bid = np.multiply(y_pred**2, base_bid/avgCTR)
    return bid

In [15]:
def max_bid_base_q(prices, y_p_valid, avgCTR):
    ration = np.zeros(len(prices))
    total = len(valid_df)

    for i in range(len(prices)):
        tmp = calculate_bid_q(prices[i], y_p_valid, avgCTR) # calculates our bid price
        tmp_2 = valid_df.click[valid_df.bidprice < tmp] # stores 0/1 for those auctions we won

        if len(tmp_2) == 0:
            ration[i] = 0
        else:
            ration[i] = sum(tmp_2)/len(tmp_2) # clicked/won actions

    print("The bid_base with best CTR: ", prices[ration.argmax()])
    print("CTR for winning bid_base is ", ration.max())
    
    return prices[ration.argmax()]

In [17]:
prices_q = range(100, 45000, 1)

bid_base_q = max_bid_base_q(prices_q, y_p_valid, avgCTR)
predicted_bid_v_q = calculate_bid_q(bid_base_q, y_p_valid, avgCTR)

imps_q = valid_df.bidprice < predicted_bid_v_q
clicks_q = valid_df.click[imps_q == 1]
total_cost_q = sum(valid_df.payprice[imps_q])

print("Cost for valid set ", total_cost_q, ", CPC ", (total_cost_q/sum(clicks_q)), " impressions ",\
      sum(imps_q)," clicks ", sum(clicks_q))

The bid_base with best CTR:  1904
CTR for winning bid_base is  0.012987012987
Cost for valid set  39816 , CPC  6636.0  impressions  462  clicks  6


In [18]:
test_results_budget(imps_q, valid_df.payprice, valid_df.click)

Sum payed for the clicked ones  6340  successful clicks  1  out of  76
Cost per Click  6340.0
CTR  0.013157894736842105


Test set

In [ ]:
# preprocess validation set
test_df['area'] = test_df['slotwidth']*test_df['slotheight']

X_test = test_df[features]

In [ ]:
## predicts probabilities of being clicked for validation
y_p_test = clf.predict(X_test)

In [ ]:
avgCTR_test = sum(y_p_test)/len(y_p_test)
print("Average CTR for test datatset ", avgCTR_test)

In [ ]:
bid_base_test = max_bid_base(prices_q, y_p_test, avgCTR_test)
predicted_bid_t_q = calculate_bid_q(bid_base_test, y_p_test, avgCTR_test)